El objetivo de este código es implementar el algoritmo GBM.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [18]:
#!pip install comet_ml

# Importo las librerías
from lightgbm import LGBMClassifier
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from comet_ml import Experiment

# Levanto los datos
features = load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50.joblib")
label = load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib")

Construyo cada conjunto de datos haciendo la división por sujeto.


*   Dos sujetos para test.
*   Dos sujetos para validación.
*   El resto para entrenamiento.

Esto lo dejé así por defecto pero puede modificarse fácilmente.

In [11]:
# Calculo la cantidad de sujetos
label = np.array(label)
cantSujetos = np.max(label[:, 2])

# Hago un sorteo a ver que sujeto va para cada conjunto
sorteo = np.random.permutation(cantSujetos) + 1

# El primer sujeto del sorteo va a test, el segundo a val y el resto a train
indices_test = list(np.where(label[:, 2]==sorteo[0])[0])
indices_val = list(np.where(label[:, 2]==sorteo[2])[0])

indices_test.extend(list(np.where(label[:, 2]==sorteo[1])[0]))
indices_val.extend(list(np.where(label[:, 2]==sorteo[3])[0]))

indices_train = []
for j in sorteo[4:]:
    indices_train.extend(np.where(label[:, 2]==j)[0])

# lo paso a numpy array para poder definir de forma mas comoda los conjuntos de train test y val
features = np.array(features)

X_train = features[indices_train, :]
y_train = label[indices_train, 1]
X_val = features[indices_val, :]
y_val = label[indices_val, 1]
X_test =  features[indices_test, :]
y_test = label[indices_test, 1]

Voy a usar la implementación que me dio ChatGPT, ya que en la tesis de Renato no se habla de cómo elegir los parámetros

In [20]:
# Crear una instancia de LGBMClassifier
clf = LGBMClassifier()

# Entrenar el clasificador
clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss')

# Realizar predicciones en el conjunto de validación
y_pred = clf.predict(X_val)

# Calcular la precisión
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy}')

# Obtener las iteraciones del mejor modelo
best_iteration = clf.best_iteration_

# Imprimir la advertencia si existe
if hasattr(clf, 'best_iteration_') and clf.best_iteration_ is None:
    print("[LightGBM] [Warning] No further splits with positive gain, best gain: -inf")



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.216863 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50308
[LightGBM] [Info] Number of data points in the train set: 37655, number of used features: 240
[LightGBM] [Info] Start training from score -2.565481
[LightGBM] [Info] Start training from score -2.592793
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.591020
[LightGBM] [Info] Start training from score -2.573805
[LightGBM] [Info] Start training from score -2.543614
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.376846
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.740986
Accurac

In [22]:
# Realizar predicciones en el conjunto de validación
y_pred = clf.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.22978754794924758


Voy a guardar el clasificador, y evaluar distintas métricas: accuracy, precision y recall. Voy a hacer una matriz de confusión.
Por otra parte, voy a guardar la partición de los datos para hacer reproducible el experimento.

In [ ]:
# Guardar el modelo entrenado en un archivo
joblib.dump(bst, 'baseline_gbm_sep_sub_r1.pkl')

# Predecir en el conjunto de test
y_pred = bst.predict(X_test)

# Calcular métricas de desempeño
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar las métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Guardar la imagen de la matriz de confusión
plt.savefig("confusion_matrix.png")

In [14]:
# Predecir en el conjunto de validación
y_pred = bst.predict(X_val, num_iteration=bst.best_iteration)

# Puedes evaluar el rendimiento del modelo utilizando diferentes métricas como precisión, recall, f1-score, etc.
# Por ejemplo, para calcular la precisión:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred.round())
print(f'Accuracy: {accuracy}')

Accuracy: 0.07803229440480661
